In [5]:
import pandas as pd

df = pd.read_csv("data.csv")
df = df[df["IsPushLap"] == 1]

GROUP_COLS = [
    "Event",
    "CircuitName",
    "Country",
    "TrackType",
    "LapSpeedClass"
]

NUMERIC_COLS = [
    "SpeedI1", "SpeedI2", "SpeedFL", "SpeedST",
    "TrackLength_m", "NumCorners", "CornerDensity",
    "AvgCornerSpacing_m", "AirTemp", "TrackTemp",
    "WindSpeed", "Altitude_m", "DRSZones"
]

circuit_medians = (
    df.groupby(GROUP_COLS)[NUMERIC_COLS]
      .median()
      .reset_index()
)

circuit_medians.to_csv("circuit_medians.csv", index=False)

print("✅ circuit_medians.csv rebuilt correctly")

✅ circuit_medians.csv rebuilt correctly


In [7]:
import pandas as pd
from catboost import CatBoostRegressor

model = CatBoostRegressor()
model.load_model("quali_model.cbm")

medians = pd.read_csv("circuit_medians.csv")

def predict_quali_time(
    driver,
    team,
    event,
    quali_segment,
    compound="SOFT",
    session="Q"
):
    row = medians[medians["Event"] == event]

    if row.empty:
        raise ValueError(f"No circuit data found for event: {event}")

    row = row.iloc[0]

    input_data = {
        "Driver": driver,
        "Team": team,
        "Compound": compound,
        "Event": event,
        "Session": session,
        "QualiSegment": quali_segment,

        "CircuitName": row["CircuitName"],
        "Country": row["Country"],
        "TrackType": row["TrackType"],
        "LapSpeedClass": row["LapSpeedClass"],

        "TyreLife": 2,
        "SpeedI1": row["SpeedI1"],
        "SpeedI2": row["SpeedI2"],
        "SpeedFL": row["SpeedFL"],
        "SpeedST": row["SpeedST"],
        "TrackLength_m": row["TrackLength_m"],
        "NumCorners": row["NumCorners"],
        "CornerDensity": row["CornerDensity"],
        "AvgCornerSpacing_m": row["AvgCornerSpacing_m"],
        "AirTemp": row["AirTemp"],
        "TrackTemp": row["TrackTemp"],
        "WindSpeed": row["WindSpeed"],
        "Altitude_m": row["Altitude_m"],
        "DRSZones": row["DRSZones"],
    }

    X = pd.DataFrame([input_data])
    return round(model.predict(X)[0], 3)

print(
    predict_quali_time(
        driver="VER",
        team="Red Bull Racing",
        event="Australian Grand Prix",
        quali_segment="Q3"
    )
)


77.644
